In [1]:
import numpy as np
import matplotlib.pyplot as plt
import QuantLib as ql

In [2]:
schedule = ql.MakeSchedule(ql.Date(15,6,2020), ql.Date(15,6,2022), ql.Period('1Y'))
putCallSchedule = ql.CallabilitySchedule()

my_price  = ql.BondPrice(100, ql.BondPrice.Clean)

putCallSchedule.append(
    ql.Callability(my_price, ql.Callability.Call, ql.Date(15,6,2021))
)

bond = ql.CallableFixedRateBond(2, 100, schedule, [0.01], ql.Actual360(), ql.ModifiedFollowing, 100, ql.Date(15,6,2020), putCallSchedule)

In [5]:
side = ql.Protection.Seller
nominal = 10e6
spread = 34.6 / 10000
cdsSchedule = ql.MakeSchedule(ql.Date(20, 12, 2019), ql.Date(20, 12, 2024), ql.Period('3M'),
                            ql.Quarterly, ql.TARGET(), ql.Following, ql.Unadjusted, ql.DateGeneration.TwentiethIMM)

cds = ql.CreditDefaultSwap(side, nominal, spread, cdsSchedule, ql.Following, ql.Actual360())

In [6]:
expiry = ql.Date(15,6,2020)
exercise = ql.EuropeanExercise(expiry)
ql.CdsOption(cds, exercise, True)

<QuantLib.QuantLib.CdsOption; proxy of <Swig Object of type 'ext::shared_ptr< CdsOption > *' at 0x1272dbde0> >